In [1]:
import isort
import sys
import platform
import importlib
from types import ModuleType

In [2]:
# module_names = isort.stdlibs.py38.stdlib

In [3]:
def get_stdlib_packages():
    if sys.version_info.minor == 10:
        module_names = sys.stdlib_module_names
    else:
        module_names = isort.stdlibs.py38.stdlib

    external_packages = set()
    for name in module_names:
        if name[0] == '_' or name == 'this' or name == 'antigravity':
            continue
        external_packages.add(name)
    return external_packages

In [4]:
def task1():
    external_packages = get_stdlib_packages()
    v = sys.version_info
    print(f'Python {v.major}.{v.minor}.{v.micro} on {platform.system()} {platform.release()}')
    print(f'Stdlib contains {len(external_packages)} external modules and packages: ')
    print(external_packages)

In [5]:
task1()

Python 3.8.10 on Linux 5.15.0-50-generic
Stdlib contains 212 external modules and packages: 
{'pprint', 'logging', 'keyword', 'resource', 'pstats', 'fractions', 'math', 'xdrlib', 'telnetlib', 'runpy', 'tokenize', 'crypt', 'fcntl', 'numbers', 'tarfile', 'linecache', 'fileinput', 'warnings', 'winsound', 'sysconfig', 'ensurepip', 'pickle', 'os', 'hmac', 'syslog', 'cgi', 'symbol', 'socket', 'formatter', 'base64', 'builtins', 'plistlib', 'spwd', 'functools', 'cmath', 'platform', 'zipimport', 'mailcap', 'locale', 'signal', 'threading', 'array', 'pty', 'asyncore', 'faulthandler', 'ossaudiodev', 'random', 'token', 'imaplib', 'zlib', 'select', 'traceback', 'venv', 'uu', 'posix', 'heapq', 'netrc', 'code', 'decimal', 'curses', 'mmap', 'asyncio', 'pkgutil', 'poplib', 'cgitb', 'contextlib', 'py_compile', 'pydoc', 'abc', 'nntplib', 'inspect', 'multiprocessing', 'site', 'sre', 'binascii', 'bdb', 'statistics', 'ftplib', 'ntpath', 'readline', 'msilib', 'chunk', 'argparse', 'stat', 'fnmatch', 'socketser

In [6]:
def get_real_packages(package_names):
    real_modules = set()
    not_importable_modules = set()
    for name in package_names:
        try:
            importlib.import_module(name)
            # del sys.modules[name]
            real_modules.add(name)
        except:
            not_importable_modules.add(name)
    return real_modules, not_importable_modules

In [7]:
def get_real():
    external_packages = get_stdlib_packages()
    return get_real_packages(external_packages)

In [8]:
def task2():
    v = sys.version_info
    real_modules, not_importable_modules = get_real()
    print(f'These StdLib packages on Python-{v.major}.{v.minor}.{v.micro}/{platform.system()} {platform.release()} '
          f'are not importable:')
    print(not_importable_modules)

    for name in real_modules:
        del sys.modules[name]

In [9]:
task2()

These StdLib packages on Python-3.8.10/Linux 5.15.0-50-generic are not importable:
{'msilib', 'sre', 'winsound', 'winreg', 'turtledemo', 'ensurepip', 'turtle', 'msvcrt', 'tkinter'}


In [10]:
def module_dependency(module_names, name):
    if name not in module_names:
        raise Exception(f'{name} is not importable module')
    dp_names = set()

    try:
        importlib.import_module(name)
    except:
        print(f'err name: {name}')
    for key, val in vars(sys.modules[name]).items():
        if isinstance(val, ModuleType):
            md_name = val.__name__

            try:
                index = md_name.index(".")
                md_name = md_name[0:index]
            except:
                pass

            dp_names.add(md_name)
            # print(f'key: {key}, type: {type(val)}, val: {md_name}, module: {md_name}')

    # del sys.modules[name]

    return dp_names

In [11]:
def core_modules(real_modules):
    # real_modules, _ = get_real()
    core_module_names = set()
    for r_name in real_modules:
        dp_names = module_dependency(real_modules, r_name)
        if len(dp_names) == 0:
            core_module_names.add(r_name)

    return core_module_names


In [12]:
def most_dependent_modules(real_modules):
    dp_dict = dict((name, 0) for name in real_modules)
    for md_name in real_modules:
        count = len(module_dependency(real_modules, md_name))
        dp_dict[md_name] = count

    sorted_dp_dict = dict(sorted(dp_dict.items(), key=lambda item: item[1], reverse=True))
    i = 0
    most_dependent_module_names = dict()
    for k, v in sorted_dp_dict.items():
        if i > 5:
            break
        most_dependent_module_names[k] = v
        i = i + 1

    return most_dependent_module_names



In [13]:
def task3():
    real_modules, not_importable_modules = get_real()
    # print(f'real modules: {real_modules}')
    core_module_names = core_modules(real_modules)

    dp_dict = most_dependent_modules(real_modules)

    print(f'The following StdLib packages are most dependent:')
    for k, v in dp_dict.items():
        print(f'{k}: {v}')
    print(f'The {len(core_module_names)} core packages are:')
    print(core_module_names)

    for name in real_modules:
        del sys.modules[name]

In [14]:
task3()

The following StdLib packages are most dependent:
gzip: 8
pathlib: 8
shutil: 7
socket: 6
zipimport: 6
re: 6
The 67 core packages are:
{'struct', 'urllib', 'keyword', 'zlib', 'select', 'stringprep', 'gc', 'sndhdr', 'resource', 'copy', 'ast', 'tty', 'math', 'posix', 'heapq', 'bisect', 'quopri', 'decimal', 'termios', 'fcntl', 'numbers', 'lib2to3', 'copyreg', 'mmap', 'sys', 'grp', 'colorsys', 'http', 'types', 'abc', 'syslog', 'unicodedata', 'parser', 'audioop', 'symbol', 'marshal', 'xmlrpc', 'binascii', 'xml', 'builtins', 'sre_parse', 'email', 'imghdr', 'time', 'concurrent', 'spwd', 'functools', 'cmath', 'errno', 'itertools', 'test', 'readline', 'operator', 'chunk', 'atexit', 'stat', 'sre_constants', 'array', 'nis', 'difflib', 'contextvars', 'pwd', 'faulthandler', 'ossaudiodev', 'unittest', 'wsgiref', 'token'}


In [37]:
def explore_package(package_name):
    importlib.import_module(package_name)
    pack = sys.modules[package_name]
    file_name = 'builtin_binary'
    try:
        file_name = pack.__file__
        n = len(file_name)
        # print(file_name[n-3:n])
        if file_name[n-3:n] == '.py':
            # print(file_name)
            return package_name, file_name, 0
        elif file_name[n-3:n] == '.so':
            # print(file_name)
            return package_name, file_name, 1
        else:
            raise Exception(f'unsupported file: {file_name}')
    except:
        return package_name, file_name, 2

p, f, r = explore_package('locale')
print(r)

0


In [38]:
real_modules, _ = get_real()

for md in real_modules:
    p, f, r = explore_package(md)
    print(f'package name: {p}, file: {f}, type: {r}')

package name: pprint, file: /usr/lib/python3.8/pprint.py, type: 0
package name: keyword, file: /usr/lib/python3.8/keyword.py, type: 0
package name: resource, file: /usr/lib/python3.8/lib-dynload/resource.cpython-38-x86_64-linux-gnu.so, type: 1
package name: pstats, file: /usr/lib/python3.8/pstats.py, type: 0
package name: fractions, file: /usr/lib/python3.8/fractions.py, type: 0
package name: math, file: builtin_binary, type: 2
package name: xdrlib, file: /usr/lib/python3.8/xdrlib.py, type: 0
package name: telnetlib, file: /usr/lib/python3.8/telnetlib.py, type: 0
package name: fcntl, file: builtin_binary, type: 2
package name: numbers, file: /usr/lib/python3.8/numbers.py, type: 0
package name: fileinput, file: /usr/lib/python3.8/fileinput.py, type: 0
package name: pickle, file: /usr/lib/python3.8/pickle.py, type: 0
package name: os, file: /usr/lib/python3.8/os.py, type: 0
package name: syslog, file: builtin_binary, type: 2
package name: symbol, file: /usr/lib/python3.8/symbol.py, type:

In [21]:
import math
print(math.__file__)
print()


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/linhnm/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_193834/1838482550.py", line 2, in <cell line: 2>
    math.__file__
AttributeError: module 'math' has no attribute '__file__'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/linhnm/.local/lib/python3.8/site-packages/executing/executing.py", line 312, in executing
    args = executing_cache[key]
KeyError: (<code object run_code at 0x7f762516b500, file "/home/linhnm/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3362>, 140145405113600, 74)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/linhnm/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 1993, in showtraceback
    stb = 

In [22]:
import pprint
print(pprint.__file__)
print(pprint.__path__)

/usr/lib/python3.8/pprint.py
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/linhnm/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_193834/3619697001.py", line 3, in <cell line: 3>
    print(pprint.__path__)
AttributeError: module 'pprint' has no attribute '__path__'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/linhnm/.local/lib/python3.8/site-packages/executing/executing.py", line 312, in executing
    args = executing_cache[key]
KeyError: (<code object run_code at 0x7f762516b500, file "/home/linhnm/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3362>, 140145405113600, 74)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/linhnm/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 1993, in showtraceback